In [61]:
import requests
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc.stop()

sc = SparkContext("local", "NHL Games").getOrCreate()
sqlContext = SQLContext(sc)
r = requests.get('https://statsapi.web.nhl.com/api/v1/schedule')
data = r.json()
games = data['dates'][0]['games']
parsed_games = list()


for game in games:
    game_r = requests.get(f'https://statsapi.web.nhl.com/{link}')
    game_data = game_r.json()
    
    parsed_game = dict({
        "game_id": game['gamePk'],
        "api_link": link,
        "game_type": game['gameType'],
        "season": game['season'],
        "timestamp":  game_data['metaData']['timeStamp'],
        "game_data": game_data['gameData'],
        "live_data":  game_data['liveData']
    })
    
    parsed_games.append(parsed_game)


rdd = sc.parallelize(parsed_games)
df = sqlContext.createDataFrame(rdd)
df.coalesce(1).write.format('json').save('/home/lucsomers/test.json')